<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Naas - Send notification

**Tags:** #naas #emailbuilder #snippet #operations

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

**Description:** This notebook send a notification with your latest posts performance.

## Input

### Import libraries

In [ ]:
from naas_drivers import emailbuilder, gsheet
import naas
import os
import naas_data_product
from datetime import date, datetime, timedelta
from naas_drivers import naasauth

### Setup variables
**Inputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `input_dir`: Input directory to retrieve file from.
- `datalake_dir`: Datalake directory to retrieve plugin json file.

**Outputs**
- `email_to`: This variable is used for storing a list of email addresses that will receive the notification email. In this case, the list contains the email address of the script user, which is retrieved using the naasauth.connect().user.me().get("username") method.
- `subject`: This variable is used for storing the subject of the notification email. In this case, the subject is "🖼️ Content Engine: LinkedIn profile update".

In [ ]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "growth-engine", date.today().isoformat())
input_file = "growth_trend"
input_image = "growth_trend.png"
input_html = "growth_trend.html"
datalake_dir = naas_data_product.OUTPUTS_PATH

# Outputs
email_to = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "emails") or []
email_subject = f"🚀 Growth Assistant - Email Update, {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

## Model

### Get assets

In [ ]:
image_url = get_image_asset(input_dir, input_image)
print("Image URL:", image_url)

image_html = get_image_asset(input_dir, input_html)
print("Image HTML:", image_html)

plugin_url = naas.asset.find(os.path.join(datalake_dir, "plugins", "🚀_growth_assistant.json"))
print("Plugin URL:", plugin_url)

### Get data from Google Sheets spreadsheet

In [ ]:
# Get data
df_init = pload(input_dir, input_file) 

# Filter on this week and last week
tw = date.today().strftime("W%W-%Y")
lw = (date.today() - timedelta(days=date.today().weekday() + 7)).strftime("W%W-%Y")
df_init = df_init[df_init["LABEL"].isin([tw, lw])]

# Display result
print("Rows:", len(df_init))
df_init.head(10)

### Prep values for email

In [ ]:
# Get data
df_init = pload(input_dir, input_file) 

# Groupby weeks
df = df_init.groupby(["LABEL", "LABEL_D"], as_index=False).agg({"VALUE": "sum"})
df = df.sort_values(by="LABEL", ascending=False).reset_index(drop=True)
df

def format_number(num):
    num = str("{:,.0f}".format(num)).replace(",", " ")
    return num

kpis = []
for row in df.itertuples():
    index = row.Index
    label = row.LABEL_D
    value = row.VALUE
    varv = 0
    if index + 1 < len(df):
        varv = value - df.loc[index + 1, "VALUE"]
    varv_d = f" ({format_number(varv)} vs last week)"
    if varv == 0:
        varv_d = ""
    kpis.append(f"{label} : <b>{format_number(value)}{varv_d}</b>")
print(kpis)

### Create Email brief

In [ ]:
def email_brief(
    kpis,
    plugin_url,
    image_url,
    image_html,
):
    content = {
        'title': "🚀 Growth Assistant - Email Update",
        'heading': (f"Date: {date.today().isoformat()}"),
        "txt_intro": (
            f"Hi there,<br><br>" f"Here is your growth engine email as of {date.today().isoformat()}."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="27px", text_align="center", bold=True
        ),
        "image_1": emailbuilder.image(image_url, image_html),
        "text_1": emailbuilder.text(
            "Here are your content indicators stats vs last week:"
        ),
        "list_1": emailbuilder.list(kpis),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin_url}"),
            text="Start Chatting With Assistant",
            background_color="#181a1c",
        ),
        "footer_cs": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = email_brief(
    kpis,
    plugin_url,
    image_url,
    image_html
)

## Output

### Send the email

In [ ]:
naas.notification.send(
    email_to=email_to,
    subject=email_subject,
    html=email_content,
)